## RFInder: tools to investigate the presence of RFI in a dataset


In [1]:
import os, string, sys
import numpy as np
import pyaml, json
from astropy.io import ascii

from IPython.display import HTML, display

import rfinder as rfinder

print '\t+------+\n\t  Done \n\t+------+'


Bad key "backed" on line 1 in
/Users/maccagni/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


	+------+
	  Done 
	+------+


### Load and modify parameter file

In [2]:
print '\t+------+\n\t RFInder \n\t Parameter File \n\t+------+'

rfi_par = rfinder.rfinder()

print '\t+------+\n\t Parameters Loaded \n\t+------+'

#print parameter file 
print(json.dumps(rfi_par.cfg_par, indent=4, sort_keys=True))


	+------+
	 RFInder 
	 Parameter File 
	+------+
	+------+
	 Parameters Loaded 
	+------+
{
    "beam_shape": {
        "__helpstr": "estimate beam shape after RFI flagging", 
        "enable": false
    }, 
    "general": {
        "__helpstr": "General INPUT directories and filenames", 
        "msfullpath": "/Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS", 
        "msname": [
            "WSRTA180824014_B000.MS"
        ], 
        "plotdir": "/Users/maccagni/Projects/RFI/data/RFI25_180824014/rfi/plots/", 
        "rfidir": "/Users/maccagni/Projects/RFI/data/RFI25_180824014/rfi/", 
        "tabledir": "/Users/maccagni/Projects/RFI/data/RFI25_180824014/rfi/tables/", 
        "verbose": false, 
        "workdir": "/Users/maccagni/Projects/RFI/data/RFI25_180824014/"
    }, 
    "plots": {
        "__helpstr": "plot rfi over different axes", 
        "enable": true, 
        "plot_long_short": true, 
        "plot_noise": "flag"
    }, 
    "rfi": {
        "_

In [3]:
# Set working directory & filenames
#rfi_par.cfg_par['general']['workdir'] = '/home/maccagni/RFI/'

# Enable / disable modules
rfi_par.cfg_par['rfi']['enable'] = True
rfi_par.cfg_par['beam_shape']['enable'] = True
rfi_par.cfg_par['plots']['enable'] = True

#MSFILE
#rfi_par.cfg_par['general']['msname'] = ['WSRTA090807_B000.MS']

rfi_par.set_dirs()

# Options
#create also tables of data binned in frequency
rfi_par.cfg_par['spw_average']['enable'] = True
rfi_par.cfg_par['spw_average']['spw_width'] = 3 #MHz
#set separation in long and short baselines
rfi_par.cfg_par['rfi']['baselines_cut'] = 615 #MHz




print '\t+------+\n\t Parameters modified \n\t+------+'


	+------+
	 Parameters modified 
	+------+


### Investigate presence of RFI in a dataset

In [4]:
import rfi
import rfinder_plots as rfi_pl

rfi = rfi.rfi()

#read info from MSfile 
rfi.load_from_ms(rfi_par.cfg_par,0)
#sort visibilities by baseline lenght
rfi.baselines_from_ms(rfi_par.cfg_par)
#flag bad antennas (from configuration file)
datas = rfi.priors_flag(rfi_par.cfg_par)
#find rfi above threshold
rfi.find_rfi(datas,rfi_par.cfg_par,-1)
rfi_pl.rfi_frequency(rfi_par.cfg_par,-1)


print '\t+------+\n\t Done \n\t+------+'


Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS/ANTENNA: 8 columns, 12 rows
	Total number of antennas	:12
	Antenna names	[ 0  1  2  3  4  5  6  7  8  9 10 11]
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS/SPECTRAL_WINDOW: 14 columns, 1 rows
	Bandwidth [kHz]	:12.20703125
	Start Frequency [GHz]	:1.249609375
	End Frequency [GHz]	:1.44959716797
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS: 23 columns, 18330 rows
... info from MS loaded ...

... baselines loaded...

... prior flagging done...

... RFI found...

	+------+
	 Done 
	+------+


### Plot RFI

In [5]:
import rfi
import rfinder_plots as rfi_pl

rfi = rfi.rfi()
#plot spectra
#read info from MSfile 
rfi.load_from_ms(rfi_par.cfg_par,0)
#sort visibilities by baseline lenght
rfi.baselines_from_ms(rfi_par.cfg_par)
rfi_pl.plot_rfi_imshow(rfi_par.cfg_par,-1)
#rfi_pl.rfi_frequency(rfi_par.cfg_par,-1)
rfi_pl.plot_noise_frequency(rfi_par.cfg_par,-1)

rfi_par.cfg_par['plots']['long_short'] = True
rfi_par.cfg_par['plots']['plot_noise'] = 'noise'

rfi_pl.plot_noise_frequency(rfi_par.cfg_par,-1)


print '\t+------+\n\t Plots done \n\t+------+'


Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS/ANTENNA: 8 columns, 12 rows
	Total number of antennas	:12
	Antenna names	[ 0  1  2  3  4  5  6  7  8  9 10 11]
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS/SPECTRAL_WINDOW: 14 columns, 1 rows
	Bandwidth [kHz]	:12.20703125
	Start Frequency [GHz]	:1.249609375
	End Frequency [GHz]	:1.44959716797
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS: 23 columns, 18330 rows
... info from MS loaded ...

... baselines loaded...

Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS: 23 columns, 18330 rows
Start date: 18Aug26:02:00:16
End date: 18Aug26:04:00:04
... RFI in 2D plotted ----
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/

### Determine RFI in data every time step store it in .fits tables

In [6]:
import rfi
import rfinder_plots as rfi_pl

rfi = rfi.rfi()

rfi_par.cfg_par['time_chunks']['enable'] = True
rfi_par.cfg_par['time_chunks']['time_step'] = 10 #in minutes

rfi_par.cfg_par['plots']['long_short'] = True
rfi_par.cfg_par['plots']['plot_noise'] = 'flag'

#read info from MSfile 
#rfi.load_from_ms(rfi_par.cfg_par)
times,start,end = rfi.time_chunk(rfi_par.cfg_par)
for i in xrange(0,len(times)-1):
    
    timez = [times[i],times[i+1]] 
    rfi.load_from_ms(rfi_par.cfg_par,timez)
    #sort visibilities by baseline lenght
    rfi.baselines_from_ms(rfi_par.cfg_par)

    #flag bad antennas (from configuration file)
    datas = rfi.priors_flag(rfi_par.cfg_par)

    #find rfi above threshold
    rfi.find_rfi(datas,rfi_par.cfg_par,i)
    rfi_pl.rfi_frequency(rfi_par.cfg_par,i)

    
    rfi_pl.plot_rfi_imshow(rfi_par.cfg_par,i)

    rfi_pl.plot_noise_frequency(rfi_par.cfg_par,i)

#create FLAG column in copy of MS file
#rfi.rfi_flag(rfi_par.cfg_par)

print '\t+------+\n\t Done \n\t+------+'

Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS: 23 columns, 18330 rows
Start date: 18Aug26:02:00:16
End date: 18Aug26:04:00:04
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS/ANTENNA: 8 columns, 12 rows
	Total number of antennas	:12
	Antenna names	[ 0  1  2  3  4  5  6  7  8  9 10 11]
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS/SPECTRAL_WINDOW: 14 columns, 1 rows
	Bandwidth [kHz]	:12.20703125
	Start Frequency [GHz]	:1.249609375
	End Frequency [GHz]	:1.44959716797
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/data/RFI25_180824014/WSRTA180824014_B000.MS: 23 columns, 18330 rows
... info from MS loaded ...

... baselines loaded...

... prior flagging done...

... RFI found...

Successful readonly open of default-locked table /Users/mac

### Determine shape of PSF after RFI flagging

In [ ]:
import rfinder_beam as rfi_beam

#create FLAG column in copy of MS file
rfi.rfi_flag(rfi_par.cfg_par)
beam = rfi_beam.make_psf(rfi_par.cfg_par)

print '\t+------+\n\t Spectra extracted \n\t+------+'
